In [1]:
import QuantLib as ql
from matplotlib import pyplot as plt
import numpy as np
import math

%matplotlib inline

# Build the Process

In [3]:
todaysDate = ql.Date(15, ql.May, 2023)
ql.Settings.instance().evaluationDate = todaysDate

## Option construction
exercise = ql.AmericanExercise(todaysDate, ql.Date(17, ql.May, 2024))
payoff = ql.PlainVanillaPayoff(ql.Option.Put, 40.0)
option = ql.VanillaOption(payoff, exercise)

# Market data
underlying = ql.SimpleQuote(36.0)
dividendYield = ql.FlatForward(todaysDate, 0.00, ql.Actual365Fixed())
volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), 0.20, ql.Actual365Fixed())
riskFreeRate = ql.FlatForward(todaysDate, 0.06, ql.Actual365Fixed())

# black scholes process contains the term structures
process = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividendYield),
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.BlackVolTermStructureHandle(volatility),
)

## Build the QD Plus Engine

In [ ]:
qdplus_engine =  ql.QdPlusAmericanEngine(
    process,
    interpolationPoints = 8,
    solverType = ql.QdPlusAmericanEngine.Halley,
    eps = 1e-6,
    maxIter = None
)

In [ ]:
results = []

#### Analytic approximations
option.setPricingEngine(ql.BaroneAdesiWhaleyApproximationEngine(process))
results.append(("Barone-Adesi-Whaley", option.NPV()))

option.setPricingEngine(ql.BjerksundStenslandApproximationEngine(process))
results.append(("Bjerksund-Stensland", option.NPV()))

In [ ]:
#### Finite-difference method
timeSteps = 801
gridPoints = 800
dampingSteps = 0
schemeDesc = ql.DouglasScheme()
fd_engine = ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints, dampingSteps, schemeDesc)

option.setPricingEngine(fd_engine)
results.append(("finite differences", option.NPV()))

# Linear Regression

In [ ]:
ql.LsmBasisSystem.Monomial 
ql.LsmBasisSystem.Laguerre
ql.LsmBasisSystem.Hermite
ql.LsmBasisSystem.Hyperbolic
ql.LsmBasisSystem.Chebyshev2nd

coeff_[i-1] = GeneralLinearLeastSquares(x, y, v_).coefficients()


In [3]:
# Longstaff Schwartz
mc_engine = ql.MCAmericanEngine(
    process,
    "pseudorandom",
    timeSteps=timeSteps,
    timeStepsPerYear=None,
    antitheticVariate=False,
    controlVariate=False,
    requiredSamples=None,
    requiredTolerance=0.02,
    maxSamples=None,
    seed=42,
    polynomOrder=3,
    polynomType=ql.LsmBasisSystem.Monomial,
    antitheticVariateCalibration=None,
    seedCalibration=None
)

option.setPricingEngine(mc_engine)

results.append(("LS American", option.NPV()))

[('Barone-Adesi-Whaley', 4.463536264085715),
 ('Bjerksund-Stensland', 4.4569588667986935),
 ('finite differences', 4.489992153451754),
 ('QD+', 4.501016922307475),
 ('QD+ fixed point', 4.490552145592267)]

In [ ]:
# #### Li, M. QD+ American engine
qdplus_engine =  ql.QdPlusAmericanEngine(
    process,
    interpolationPoints = 8,
    solverType = ql.QdPlusAmericanEngine.Halley,
    eps = 1e-6,
    maxIter = None
)
option.setPricingEngine(qdplus_engine)
results.append(("QD+", option.NPV()))

In [ ]:
# #### Leif Andersen, Mark Lake and Dimitri Offengenden high performance American engine

# Gauss-Legendre (l,m,n)-p Scheme
#        \param l  order of Gauss-Legendre integration within every fixed point iteration step
#        \param m  fixed point iteration steps, first step is a partial Jacobi-Newton,
#                  the rest are naive Richardson fixed point iterations
#        \param n  number of Chebyshev nodes to interpolate the exercise boundary
#        \param p  order of Gauss-Legendre integration in final conversion of the
#                  exercise boundary into option prices

ql.QdFpIterationScheme, ql.QdFpLegendreScheme, ql.QdFpLegendreTanhSinhScheme, ql.QdFpTanhSinhIterationScheme
# ql.QdFpAmericanEngine.fastScheme(), ql.QdFpAmericanEngine.highPrecisionScheme()

qdf_engine = ql.QdFpAmericanEngine(process, ql.QdFpAmericanEngine.accurateScheme()) 

option.setPricingEngine(qdf_engine)

results.append(("QD+ fixed point", option.NPV()))

## Build Iteration Scheme

In [ ]:
# ql.QdFpLegendreTanhSinhScheme, ql.QdFpTanhSinhIterationScheme
iteration_scheme = ql.QdFpLegendreScheme(1,2,3,4)

In [ ]:
def calculatePut(S, K, r, q, vol, T, iteration_scheme, process, QdPlusAmericanEngine) -> float:
   
    if (r < 0.0) and (q < r):
        raise Exception, "double-boundary case q<r<0 for a put option is given"

    xmax = QdPlusAmericanEngine.xMax(K, r, q)
    n = iteration_scheme.getNumberOfChebyshevInterpolationNodes()

    interp: ql.ChebyshevInterpolation = ql.QdPlusAmericanEngine(
        process, 
        n+1, 
        ql.QdPlusAmericanEngine.Halley, 
        1e-8
    ).getPutExerciseBoundary(S, K, r, q, vol, T)

    z: ql.Array = interp.nodes()
    x: ql.Array = 0.5*np.sqrt(T)*(1.0+z)

    # Boundary
    B = lambda xmax, T, interp, tau: xmax * np.exp(-np.sqrt(np.max(0, interp(2*np.sqrt(np.abs(tau)/T)-1, True))))

    h = lambda fv, xmax : np.squared(np.log(fv/xmax))
        
    # eqn: ql.DqFpEquation = (fpEquation_ == FP_A || (fpEquation_ == Auto && np.abs(r-q) < 0.001)) ? DqFpEquation_A(K, r, q, vol, B, iterationScheme_->getFixedPointIntegrator()) : DqFpEquation_B(K, r, q, vol, B, iterationScheme_->getFixedPointIntegrator()))

    eqn = ql.DqFpEquation_A(K, r, q, vol, B, iteration_scheme.getFixedPointIntegrator())
    y = ql.Array(x.size())
    y[0] = 0.0

    n_newton = iteration_scheme.getNumberOfJacobiNewtonFixedPointSteps()
    for k in range(n_newton):
        for i in range(1, x.size()):
            tau = np.square(x[i])
            b = B(tau)
            
            N, D, fv = eqn.f(tau, b)
           
            if (tau < 0.0001):
                y[i] = h(fv)
            else:
                Nd, Dd = eqn.NDd(tau, b)
                # Nd = std::get<0>(ndd)
                # Dd = std::get<1>(ndd)
                fd = K * np.exp(-(r-q) * tau) * (Nd/D - Dd*N/(D*D))
                y[i] = h(b - (fv - b)/ (fd-1))
        interp.updateY(y)

    n_fp = iteration_scheme.getNumberOfNaiveFixedPointSteps()
    for k in range(n_fp):
        for i in range(1, x.size()):
            tau = np.squared(x[i])
            _, fv = eqn.f(tau, B(tau))
            y[i] = h(fv)
        interp.updateY(y)
        
    # detail::QdPlusAddOnValue aov(T, S, K, r, q, vol, xmax, interp)
    # addOn = (*iteration_scheme.getExerciseBoundaryToPriceIntegrator())(aov, 0.0, np.sqrt(T))

    europeanValue = ql.BlackCalculator(ql.Option.Put, K, S * np.exp((r-q)*T), vol*np.sqrt(T), np.exp(-r*T)).value()

    return np.max(europeanValue, 0.0) + np.max(0.0, addOn)



In [ ]:
class DqFpEquation(
     def __init__(self)
     Rate _r, Rate _q, Volatility _vol, std::function<Real(Real)> B, ext::shared_ptr<Integrator> _integrator)
        : r(_r), q(_q), vol(_vol), B(std::move(B)), integrator(std::move(_integrator)) {
            const auto legendreIntegrator =
                ext::dynamic_pointer_cast<GaussLegendreIntegrator>(integrator);

            if (legendreIntegrator != nullptr) {
                x_i = legendreIntegrator->getIntegration()->x();
                w_i = legendreIntegrator->getIntegration()->weights();
            }
        }

        virtual std::pair<Real, Real> NDd(Real tau, Real b) const = 0;
        virtual std::tuple<Real, Real, Real> f(Real tau, Real b) const = 0;

        std::pair<Real, Real> d(Time t, Real z) const {
            const Real v = vol * std::sqrt(t);
            const Real m = (std::log(z) + (r-q)*t)/v + 0.5*v;
            return std::make_pair(m, m-v);
        }

        Array x_i, w_i
        const Rate r, q
        const Volatility vol

        std::function<Real(Real)> B
        ext::shared_ptr<Integrator> integrator

        self.phi = ql.NormalDistribution()
        self.Phi = ql.CumulativeNormalDistribution()

    
    
class DqFpEquation_B: public DqFpEquation {
      public:
        DqFpEquation_B(Real K,
                       Rate _r,
                       Rate _q,
                       Volatility _vol,
                       std::function<Real(Real)> B,
                       ext::shared_ptr<Integrator> _integrator);

        std::pair<Real, Real> NDd(Real tau, Real b) const override;
        std::tuple<Real, Real, Real> f(Real tau, Real b) const override;

      private:
          const Real K;
    };

    class DqFpEquation_A: public DqFpEquation {
      public:
        DqFpEquation_A(Real K,
                       Rate _r,
                       Rate _q,
                       Volatility _vol,
                       std::function<Real(Real)> B,
                       ext::shared_ptr<Integrator> _integrator);

        std::pair<Real, Real> NDd(Real tau, Real b) const override;
        std::tuple<Real, Real, Real> f(Real tau, Real b) const override;

      private:
          const Real K;
    };

    DqFpEquation_A::DqFpEquation_A(Real K,
                                   Rate _r,
                                   Rate _q,
                                   Volatility _vol,
                                   std::function<Real(Real)> B,
                                   ext::shared_ptr<Integrator> _integrator)
    : DqFpEquation(_r, _q, _vol, std::move(B), std::move(_integrator)), K(K) {}

    std::tuple<Real, Real, Real> DqFpEquation_A::f(Real tau, Real b) const {
        const Real v = vol * std::sqrt(tau);

        Real N, D;
        if (tau < squared(QL_EPSILON)) {
            if (close_enough(b, K)) {
                N = 1/(M_SQRT2*M_SQRTPI * v);
                D = N + 0.5;
            }
            else {
                N = 0.0;
                D = (b > K)? 1.0: 0.0;
            }
        }
        else {
            const Real stv = std::sqrt(tau)/vol;

            Real K12, K3;
            if (!x_i.empty()) {
                K12 = K3 = 0.0;

                for (Integer i = x_i.size()-1; i >= 0; --i) {
                    const Real y = x_i[i];
                    const Real m = 0.25*tau*squared(1+y);
                    const std::pair<Real, Real> dpm = d(m, b/B(tau-m));

                    K12 += w_i[i] * std::exp(q*tau - q*m)
                        *(0.5*tau*(y+1)*Phi(dpm.first) + stv*phi(dpm.first));
                    K3 += w_i[i] * stv*std::exp(r*tau-r*m)*phi(dpm.second);
                }
            } else {
                K12 = (*integrator)([&, this](Real y) -> Real {
                    const Real m = 0.25*tau*squared(1+y);
                    const Real df = std::exp(q*tau - q*m);

                    if (y <= 5*QL_EPSILON - 1) {
                        if (close_enough(b, B(tau-m)))
                            return df*stv/(M_SQRT2*M_SQRTPI);
                        else
                            return 0.0;
                    }
                    else {
                        const Real dp = d(m, b/B(tau-m)).first;
                        return df*(0.5*tau*(y+1)*Phi(dp) + stv*phi(dp));
                    }
                }, -1, 1);

                K3 = (*integrator)([&, this](Real y) -> Real {
                    const Real m = 0.25*tau*squared(1+y);
                    const Real df = std::exp(r*tau-r*m);

                    if (y <= 5*QL_EPSILON - 1) {
                        if (close_enough(b, B(tau-m)))
                            return df*stv/(M_SQRT2*M_SQRTPI);
                        else
                            return 0.0;
                    }
                    else
                        return df*stv*phi(d(m, b/B(tau-m)).second);
                }, -1, 1);
            }
            const std::pair<Real, Real> dpm = d(tau, b/K);
            N = phi(dpm.second)/v + r*K3;
            D = phi(dpm.first)/v + Phi(dpm.first) + q*K12;
        }

        const Real alpha = K*std::exp(-(r-q)*tau);
        Real fv;
        if (tau < squared(QL_EPSILON)) {
            if (close_enough(b, K))
                fv = alpha;
            else if (b > K)
                fv = 0.0;
            else {
                if (close_enough(q, Real(0)))
                    fv = alpha*r*((q < 0)? -1.0 : 1.0)/QL_EPSILON;
                else
                    fv = alpha*r/q;
            }
        }
        else
            fv = alpha*N/D;

        return std::make_tuple(N, D, fv);
    }

    std::pair<Real, Real> DqFpEquation_A::NDd(Real tau, Real b) const {
        Real Dd, Nd;

        if (tau < squared(QL_EPSILON)) {
            if (close_enough(b, K)) {
                const Real sqTau = std::sqrt(tau);
                const Real vol2 = vol*vol;
                Dd = M_1_SQRTPI*M_SQRT_2*(
                    -(0.5*vol2 + r-q) / (b*vol*vol2*sqTau) + 1 / (b*vol*sqTau));
                Nd = M_1_SQRTPI*M_SQRT_2 * (-0.5*vol2 + r-q)  / (b*vol*vol2*sqTau);
            }
            else
                Dd = Nd = 0.0;
        }
        else {
            const std::pair<Real, Real> dpm = d(tau, b/K);

            Dd = -phi(dpm.first) * dpm.first / (b*vol*vol*tau) +
                    phi(dpm.first) / (b*vol * std::sqrt(tau));
            Nd = -phi(dpm.second) * dpm.second / (b*vol*vol*tau);
        }

        return std::make_pair(Nd, Dd);
    }


    DqFpEquation_B::DqFpEquation_B(Real K,
                                   Rate _r,
                                   Rate _q,
                                   Volatility _vol,
                                   std::function<Real(Real)> B,
                                   ext::shared_ptr<Integrator> _integrator)
    : DqFpEquation(_r, _q, _vol, std::move(B), std::move(_integrator)), K(K) {}


    std::tuple<Real, Real, Real> DqFpEquation_B::f(Real tau, Real b) const {
        Real N, D;
        if (tau < squared(QL_EPSILON)) {
            if (close_enough(b, K))
                N = D = 0.5;
            else if (b < K)
                N = D = 0.0;
            else
                N = D = 1.0;
        }
        else {
            Real ni, di;
            if (!x_i.empty()) {
                const Real c = 0.5*tau;

                ni = di = 0.0;
                for (Integer i = x_i.size()-1; i >= 0; --i) {
                    const Real u = c*x_i[i] + c;
                    const std::pair<Real, Real> dpm = d(tau - u, b/B(u));
                    ni += w_i[i] * std::exp(r*u)*Phi(dpm.second);
                    di += w_i[i] * std::exp(q*u)*Phi(dpm.first);
                }
                ni *= c;
                di *= c;
            } else {
                ni = (*integrator)([&, this](Real u) -> Real {
                	const Real df = std::exp(r*u);
                    if (u >= tau*(1 - 5*QL_EPSILON)) {
                        if (close_enough(b, B(u)))
                            return 0.5*df;
                        else
                            return df*((b < B(u)? 0.0: 1.0));
                    }
                    else
                        return df*Phi(d(tau - u, b/B(u)).second);
                }, 0, tau);
                di = (*integrator)([&, this](Real u) -> Real {
                	const Real df = std::exp(q*u);
                    if (u >= tau*(1 - 5*QL_EPSILON)) {
                        if (close_enough(b, B(u)))
                            return 0.5*df;
                        else
                            return df*((b < B(u)? 0.0: 1.0));
                    }
                    else
                        return df*Phi(d(tau - u, b/B(u)).first);
                    }, 0, tau);
            }

            const std::pair<Real, Real> dpm = d(tau, b/K);

            N = Phi(dpm.second) + r*ni;
            D = Phi(dpm.first) + q*di;
        }

        Real fv;
        const Real alpha = K*std::exp(-(r-q)*tau);
        if (tau < squared(QL_EPSILON)) {
            if (close_enough(b, K) || b > K)
                fv = alpha;
            else {
                if (close_enough(q, Real(0)))
                    fv = alpha*r*((q < 0)? -1.0 : 1.0)/QL_EPSILON;
                else
                    fv = alpha*r/q;
            }
        }
        else
            fv = alpha*N/D;

        return std::make_tuple(N, D, fv);
    }

    std::pair<Real, Real> DqFpEquation_B::NDd(Real tau, Real b) const {
        const std::pair<Real, Real> dpm = d(tau, b/K);
        return std::make_pair(
            phi(dpm.second) / (b*vol*std::sqrt(tau)),
            phi(dpm.first)  / (b*vol*std::sqrt(tau))
        );
    }

    QdFpAmericanEngine::QdFpAmericanEngine(
        ext::shared_ptr<GeneralizedBlackScholesProcess> bsProcess,
        ext::shared_ptr<QdFpIterationScheme> iterationScheme,
        FixedPointEquation fpEquation)
    : detail::QdPutCallParityEngine(std::move(bsProcess)),
      iterationScheme_(std::move(iterationScheme)),
      fpEquation_(fpEquation) {
    }

    ext::shared_ptr<QdFpIterationScheme>
    QdFpAmericanEngine::fastScheme() {
        static auto scheme = ext::make_shared<QdFpLegendreScheme>(7, 2, 7, 27);
        return scheme;
    }

    ext::shared_ptr<QdFpIterationScheme>
    QdFpAmericanEngine::accurateScheme() {
        static auto scheme = ext::make_shared<QdFpLegendreTanhSinhScheme>(25, 5, 13, 1e-8);
        return scheme;
    }

    ext::shared_ptr<QdFpIterationScheme>
    QdFpAmericanEngine::highPrecisionScheme() {
        static auto scheme = ext::make_shared<QdFpTanhSinhIterationScheme>(10, 30, 1e-10);
        return scheme;
    }

    Real QdFpAmericanEngine::calculatePut(
            Real S, Real K, Rate r, Rate q, Volatility vol, Time T) const {

        if (r < 0.0 && q < r)
            QL_FAIL("double-boundary case q<r<0 for a put option is given");

        const Real xmax =  QdPlusAmericanEngine::xMax(K, r, q);
        const Size n = iterationScheme_->getNumberOfChebyshevInterpolationNodes();

        const ext::shared_ptr<ChebyshevInterpolation> interp =
            QdPlusAmericanEngine(
                    process_, n+1, QdPlusAmericanEngine::Halley, 1e-8)
                .getPutExerciseBoundary(S, K, r, q, vol, T);

        const Array z = interp->nodes();
        const Array x = 0.5*std::sqrt(T)*(1.0+z);

        const auto B = [xmax, T, &interp](Real tau) -> Real {
            const Real z = 2*std::sqrt(std::abs(tau)/T)-1;
            return xmax*std::exp(-std::sqrt(std::max(Real(0), (*interp)(z, true))));
        };

        const auto h = [=](Real fv) -> Real {
            return squared(std::log(fv/xmax));
        };

        const ext::shared_ptr<DqFpEquation> eqn
            = (fpEquation_ == FP_A
               || (fpEquation_ == Auto && std::abs(r-q) < 0.001))?
              ext::shared_ptr<DqFpEquation>(new DqFpEquation_A(
                  K, r, q, vol, B,
                  iterationScheme_->getFixedPointIntegrator()))
            : ext::shared_ptr<DqFpEquation>(new DqFpEquation_B(
                    K, r, q, vol, B,
                    iterationScheme_->getFixedPointIntegrator()));

        Array y(x.size());
        y[0] = 0.0;

        const Size n_newton
            = iterationScheme_->getNumberOfJacobiNewtonFixedPointSteps();
        for (Size k=0; k < n_newton; ++k) {
            for (Size i=1; i < x.size(); ++i) {
                const Real tau = squared(x[i]);
                const Real b = B(tau);

                const std::tuple<Real, Real, Real> results = eqn->f(tau, b);
                const Real N = std::get<0>(results);
                const Real D = std::get<1>(results);
                const Real fv = std::get<2>(results);

                if (tau < QL_EPSILON)
                    y[i] = h(fv);
                else {
                    const std::pair<Real, Real> ndd = eqn->NDd(tau, b);
                    const Real Nd = std::get<0>(ndd);
                    const Real Dd = std::get<1>(ndd);

                    const Real fd = K*std::exp(-(r-q)*tau) * (Nd/D - Dd*N/(D*D));

                    y[i] = h(b - (fv - b)/ (fd-1));
                }
            }
            interp->updateY(y);
        }

        const Size n_fp = iterationScheme_->getNumberOfNaiveFixedPointSteps();
        for (Size k=0; k < n_fp; ++k) {
            for (Size i=1; i < x.size(); ++i) {
                const Real tau = squared(x[i]);
                const Real fv = std::get<2>(eqn->f(tau, B(tau)));

                y[i] = h(fv);
            }
            interp->updateY(y);
        }

        const detail::QdPlusAddOnValue aov(T, S, K, r, q, vol, xmax, interp);
        const Real addOn =
           (*iterationScheme_->getExerciseBoundaryToPriceIntegrator())(
               aov, 0.0, std::sqrt(T));

        const Real europeanValue = BlackCalculator(
            Option::Put, K, S*std::exp((r-q)*T),
            vol*std::sqrt(T), std::exp(-r*T)).value();

        return std::max(europeanValue, 0.0) + std::max(0.0, addOn);
    }

}